In [345]:
#!g1.4
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import optuna
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score as rauc
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [342]:
#!g1.4
df = pd.read_csv('https://raw.githubusercontent.com/takimov/MLTinkoff/master/train.csv',encoding='utf-8').drop(columns='Unnamed: 0')
df.loc[df['sentiment']=='+', 'sentiment'] = 1
df.loc[df['sentiment']=='−', 'sentiment'] = -1
df.loc[df['sentiment']=='?', 'sentiment'] = 0
df[['sentence','sentiment']]

,sentence,sentiment
0,При этом всегда получал качественные услуги.,1
1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",-1
2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",-1
3,Отвратительное отношение к клиентам.,-1
4,"Всегда в любое время дня и ночи помогут, ответ...",1
...,...,...
19356,Никогда и ни в коем случае не открывайте счет ...,-1
19357,ТИ откровенно забили на качество и развивают с...,-1
19358,"Я считаю, это прорыв и лидерство финансовых ус...",1
19359,"Писал мужчина очень доходчиво, не финансовым я...",1


In [343]:
#!g1.4
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)

In [344]:
#!g1.4
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Li

### Doing for 19361 sentences


In [301]:
#!g1.4
segments_ids_ans = list()
indexed_tokens_ans = list()
for i in tqdm(range(len(df['sentence']))):
    text = df['sentence'][i]
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    segments_ids = [1] * len(tokenized_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids_ans.append(segments_ids)
    indexed_tokens_ans.append(indexed_tokens)

100%|██████████| 19361/19361 [00:11<00:00, 1734.37it/s]


In [270]:
#!g1.4
sentences_count = len(df['sentence'])
tokens_tensor_ans = [torch.tensor([indexed_tokens_ans[i]]) for i in tqdm(range(sentences_count))]
segments_tensors_ans = [torch.tensor([segments_ids_ans[i]]) for i in tqdm(range(sentences_count))]

100%|██████████| 19361/19361 [00:00<00:00, 95727.61it/s]


In [ ]:
#!g1.4
encoded_layers_ans = list()
with torch.no_grad():
    for i in tqdm(range(sentences_count)):
        encoded_layers, _ = model(tokens_tensor_ans[i], segments_tensors_ans[i])
        encoded_layers_ans.append(encoded_layers)

In [138]:
#!g1.4
token_embeddings = [torch.stack(encoded_layers_ans[i], dim=0) for i in tqdm(range(sentences_count))]
token_embeddings = [torch.squeeze(token_embeddings[i], dim=1) for i in tqdm(range(sentences_count))]
token_embeddings = [token_embeddings[i].permute(1,0,2) for i in tqdm(range(sentences_count))]

100%|██████████| 19361/19361 [00:00<00:00, 325679.56it/s]


In [139]:
#!g1.4
token_vecs_sum = [[] for i in tqdm(range(sentences_count))]
for i in tqdm(range(sentences_count)):
    for token in token_embeddings[i]:
        sum_vec = torch.sum(token[-4:], dim=0)
        token_vecs_sum[i].append(sum_vec)

100%|██████████| 19361/19361 [00:15<00:00, 1245.73it/s]


In [152]:
#!g1.4
sentence_embedding_ans = list()
for i in tqdm(range(sentences_count)):
    token_vecs = encoded_layers_ans[i][11][0]
    sentence_embedding = torch.mean(token_vecs, dim=0)
    sentence_embedding_ans.append(sentence_embedding)

100%|██████████| 19361/19361 [00:02<00:00, 9137.78it/s]


In [141]:
#!g1.4
with open ('embedding.npy', 'wb') as f:
    np.save(f, np.array(sentence_embedding_ans))

In [164]:
#!g1.4
sentence_embedding_ans

array([[ 0.19534642, -0.49691603,  0.66334367, ..., -0.08865543,
        -0.04833395, -0.23987795],
       [ 0.12297825, -0.41670936,  0.18545565, ...,  0.05767892,
        -0.02469435,  0.00142835],
       [ 0.1038917 , -0.45776427,  0.41683024, ..., -0.0145911 ,
        -0.3258895 , -0.15843569],
       ...,
       [-0.01610743, -0.27199015,  0.4125188 , ...,  0.07436399,
        -0.00652051, -0.23662536],
       [ 0.37400064, -0.33378348,  0.5738513 , ...,  0.03479397,
        -0.25966772, -0.18580337],
       [-0.01634462, -0.36110577,  0.30266383, ...,  0.01920702,
        -0.18327518, -0.31647643]], dtype=float32)

### Model training + tuning

In [156]:
#!g1.4
x_train, x_test, y_train, y_test = train_test_split(sentence_embedding_ans, df['sentiment'], test_size = 0.3, train_size = 0.7, random_state  = 69)

In [146]:
#!g1.4
model = CatBoostClassifier(loss_function='MultiClass', eval_metric='AUC')

In [153]:
#!g1.4
sentence_embedding_ans = np.array([sentence_embedding_ans[i].numpy() for i in range(sentences_count)])

In [154]:
#!g1.4
with open ('sent_embedding.npy', 'wb') as f:
    np.save(f, np.array(sentence_embedding_ans))

In [157]:
#!g1.4
model.fit(x_train,y_train)

Learning rate set to 0.090376
0:	total: 121ms	remaining: 2m 1s
1:	total: 188ms	remaining: 1m 33s
2:	total: 269ms	remaining: 1m 29s
3:	total: 343ms	remaining: 1m 25s
4:	total: 419ms	remaining: 1m 23s
5:	total: 480ms	remaining: 1m 19s
6:	total: 554ms	remaining: 1m 18s
7:	total: 618ms	remaining: 1m 16s
8:	total: 694ms	remaining: 1m 16s
9:	total: 765ms	remaining: 1m 15s
10:	total: 832ms	remaining: 1m 14s
11:	total: 898ms	remaining: 1m 13s
12:	total: 958ms	remaining: 1m 12s
13:	total: 1.02s	remaining: 1m 11s
14:	total: 1.08s	remaining: 1m 11s
15:	total: 1.15s	remaining: 1m 10s
16:	total: 1.23s	remaining: 1m 10s
17:	total: 1.29s	remaining: 1m 10s
18:	total: 1.35s	remaining: 1m 9s
19:	total: 1.42s	remaining: 1m 9s
20:	total: 1.49s	remaining: 1m 9s
21:	total: 1.55s	remaining: 1m 9s
22:	total: 1.62s	remaining: 1m 8s
23:	total: 1.69s	remaining: 1m 8s
24:	total: 1.76s	remaining: 1m 8s
25:	total: 1.83s	remaining: 1m 8s
26:	total: 1.9s	remaining: 1m 8s
27:	total: 1.97s	remaining: 1m 8s
28:	total: 2

In [159]:
#!g1.4
predict = model.predict_proba(x_test)

In [160]:
#!g1.4
rauc(list(y_test), list(predict), multi_class = 'ovr')

0.9322061002818867

In [162]:
#!g1.4
def objective(trial):
    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-1),
        'depth': trial.suggest_int('depth', 2, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 1e3),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.01, 100.0),
        'random_strength': trial.suggest_float('random_strength', 1e-5, 10),
        'od_type': 'Iter',
        'od_wait': 50,
        'loss_function': 'MultiClass',
        'eval_metric': 'AUC',
        'logging_level': 'Silent',
        'task_type': 'CPU'
    }

    model = CatBoostClassifier(**params)
    model.fit(x_train, y_train, eval_set=(x_test, y_test))
    y_pred = model.predict_proba(x_test)
    accuracy = rauc(list(y_test), list(predict), multi_class = 'ovr')
    return accuracy
# Создание объекта Study для байесовской оптимизации
study = optuna.create_study(direction='maximize')


# Запуск байесовской оптимизации
study.optimize(objective, n_trials=100)

[I 2023-04-08 23:57:55,626] A new study created in memory with name: no-name-b92a9dec-8596-4aee-8268-e61db21c4ed0
[I 2023-04-08 23:57:57,480] Trial 0 finished with value: 0.9322061002818867 and parameters: {'learning_rate': 0.09985752792364147, 'depth': 7, 'l2_leaf_reg': 120.7053060762596, 'bagging_temperature': 50.607511082516766, 'random_strength': 2.301199387253877}. Best is trial 0 with value: 0.9322061002818867.
[I 2023-04-08 23:57:59,625] Trial 1 finished with value: 0.9322061002818867 and parameters: {'learning_rate': 0.00023398593722044245, 'depth': 3, 'l2_leaf_reg': 539.8801848500551, 'bagging_temperature': 55.723714957893925, 'random_strength': 3.4845325581323667}. Best is trial 0 with value: 0.9322061002818867.
[I 2023-04-08 23:58:48,992] Trial 2 finished with value: 0.9322061002818867 and parameters: {'learning_rate': 0.0570311974021931, 'depth': 5, 'l2_leaf_reg': 581.2795550136428, 'bagging_temperature': 17.393007255898915, 'random_strength': 6.033412451552875}. Best is tr

In [163]:
#!g1.4
study.best_params

{'learning_rate': 0.09985752792364147,
 'depth': 7,
 'l2_leaf_reg': 120.7053060762596,
 'bagging_temperature': 50.607511082516766,
 'random_strength': 2.301199387253877}

### Model training+testing


In [324]:
#!g1.4
model_cat = CatBoostClassifier(loss_function='MultiClass', eval_metric='AUC')

In [327]:
#!g1.4
x_test = np.load('sent_embedding.npy')

In [328]:
#!g1.4
x_train = np.load('sent_embedding.npy')

In [331]:
#!g1.4
y_train = df['sentiment']

In [332]:
#!g1.4
model.fit(x_train,y_train)

Learning rate set to 0.092036
0:	total: 178ms	remaining: 2m 57s
1:	total: 268ms	remaining: 2m 13s
2:	total: 349ms	remaining: 1m 55s
3:	total: 414ms	remaining: 1m 43s
4:	total: 483ms	remaining: 1m 36s
5:	total: 568ms	remaining: 1m 34s
6:	total: 657ms	remaining: 1m 33s
7:	total: 752ms	remaining: 1m 33s
8:	total: 836ms	remaining: 1m 32s
9:	total: 919ms	remaining: 1m 30s
10:	total: 1s	remaining: 1m 30s
11:	total: 1.09s	remaining: 1m 29s
12:	total: 1.18s	remaining: 1m 29s
13:	total: 1.27s	remaining: 1m 29s
14:	total: 1.36s	remaining: 1m 29s
15:	total: 1.45s	remaining: 1m 29s
16:	total: 1.52s	remaining: 1m 28s
17:	total: 1.59s	remaining: 1m 26s
18:	total: 1.67s	remaining: 1m 26s
19:	total: 1.75s	remaining: 1m 25s
20:	total: 1.84s	remaining: 1m 25s
21:	total: 1.93s	remaining: 1m 25s
22:	total: 2.05s	remaining: 1m 27s
23:	total: 2.15s	remaining: 1m 27s
24:	total: 2.24s	remaining: 1m 27s
25:	total: 2.33s	remaining: 1m 27s
26:	total: 2.4s	remaining: 1m 26s
27:	total: 2.48s	remaining: 1m 26s
28:	

In [333]:
#!g1.4
predict = model.predict_proba(x_test)

In [335]:
#!g1.4
with open ('predict_ans.npy', 'wb') as f:
    np.save(f, (predict))


In [340]:
#!g1.4
params = {
    'iterations': 1000,
    'learning_rate': 0.1,
    'depth': 6,
    'l2_leaf_reg': 3,
    'border_count': 32,
    'loss_function': 'MultiClass',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'verbose': False
    
}
model_cv = CatBoostClassifier(**params)
cv_scores = cross_val_score(model_cv, x_train, df['sentiment'], cv=6, scoring='roc_auc_ovr')
mean_cv_score = cv_scores.mean()
print(f'Mean ROC-AUC score: {mean_cv_score:.2f}')

Mean ROC-AUC score: 0.94


In [348]:
#!g1.4
!git remote

fatal: not a git repository (or any parent up to mount point /home/jupyter/work)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


Exception: Process exited with code 128

In [ ]:
#!g1.4
